# Library

In [1]:
#import sys
import pandas as pd
import numpy as np
import random
import pyspark
import itertools
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import pickle
import statsmodels.api as sm

from pyspark import SparkContext, SQLContext

from math import sqrt
from time import time as ttt

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier as DTC_spark
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import joblib
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import parallel_backend


# Upload

In [2]:
spark = SparkSession.builder.master("local").appName("spark_app_1234").getOrCreate()
sc = spark.sparkContext
spark
d0 = (
    spark
    .read
    .format("csv") 
    .option("header","true") 
    .option("inferSchema","true") 
    .load("gs://mas-a5-storage-1/notebooks/jupyter/application_train.csv")
)

d1 = d0.filter(d0.DAYS_EMPLOYED != 365243).select('TARGET','DAYS_EMPLOYED')

print(d1.rdd.getNumPartitions())

4


# Function

In [3]:
def prepare_spark_data(n_part, k_mult):
    '''
    takes 'DAYS_EMPLOYED and 'TARGET'  from d1 (alreay filtered)
    rearrane in n partitions (if n==0 keeps initial number of partitions)
    prints final shape/ partition
    returns d2 - spark df
    '''
    data = d1
    data_new = data # first step in the cycle
    for i in range(k_mult-1):
        data_tmp = data.select('TARGET', \
                     f.col('DAYS_EMPLOYED')*(f.lit(0.9995) + f.rand()/1000)).\
                   toDF('TARGET','DAYS_EMPLOYED')
        data_tmp = data_tmp.select('TARGET', f.floor('DAYS_EMPLOYED'))
        data_new = data_new.union(data_tmp)

    assembler = VectorAssembler(inputCols=["DAYS_EMPLOYED"], 
                        outputCol="DAYS_EMPLOYED_vect")
    d2 = assembler.transform(data_new)
    if n_part != 0:
        d2 = d2.repartition(n_part)       
    #print(f'n-partitions initial: {d2.rdd.getNumPartitions()}; df size: {d2.count()}\n')

    return d2

# Params an run

In [ ]:
## !! mind the fn NOT TO rewrite results
fn = '4cpu_by_16n__4m'
size_mult = [1, 2, 5, 10, 15, 20, 30, 50, 100]
partitions = [4, 6, 8, 16, 24, 32, 48] 
# initial, 2**i incl n_nodes till n_cpu*n_nodes and n_cpu*n_nodes
n_iter = 3

print('d1-size', d1.count())
n_part_base = d1.rdd.getNumPartitions()
print('n-partitions initial', n_part_base, '\n')

rd1 = {}
for k_size_mult in size_mult:
    rd2 = {}
    for n_part in partitions:
        df = prepare_spark_data(n_part, k_size_mult)
        df.cache()
        print('======================================================================')
        print(f'=== size_mult={k_size_mult}; \
        n_part_req={n_part}; n_part_act={df.rdd.getNumPartitions()}; df_count={df.count()} ===\n')
        times = [0 for i in range(n_iter)]
        models = {}
        for i in range(n_iter):
            dt = DTC_spark(labelCol="TARGET",
                       featuresCol="DAYS_EMPLOYED_vect",
                       minInfoGain=0.0001,
                       impurity='entropy',
                       maxDepth=14, maxBins=2**14, # it differs from scikit learn - it means number of canidate split points
                       #minInstancesPerNode = 1,
                       #checkpointInterval = 10
                       )
           
            t0 = ttt()
            model = dt.fit(df)
            t1 = ttt()
            times[i] = t1-t0
            models[i] = model 
            print('model', i, 'build time', round(times[i],2), '\n', model)
        rd2[n_part] = (models, times)
    rd1[k_size_mult] = rd2

result = rd1  

d1-size 252137
n-partitions initial 4 



=== size_mult=1;         n_part_req=4; n_part_act=4; df_count=252137 ===



model 0 build time 11.53 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_a48900d0167a, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 1 build time 8.29 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_c8e5a8300913, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 2 build time 8.06 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_6b3707428882, depth=14, numNodes=123, numClasses=2, numFeatures=1


...

model 2 build time 13.48 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_2cc685c08753, depth=11, numNodes=33, numClasses=2, numFeatures=1


=== size_mult=100;         n_part_req=48; n_part_act=48; df_count=25213700 ===



model 0 build time 22.03 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_9877d1dcd181, depth=12, numNodes=39, numClasses=2, numFeatures=1


model 1 build time 13.77 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_570df3ea1cd3, depth=12, numNodes=39, numClasses=2, numFeatures=1


model 2 build time 16.17 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_2c84c806a601, depth=12, numNodes=39, numClasses=2, numFeatures=1


# Results

In [6]:
di = result
df_res_time = pd.DataFrame()
df_res_nodes = pd.DataFrame()
for p1 in di.keys():
    for p2 in di[p1].keys():
        df_res_time.loc[p1, p2] = np.round(np.mean(di[p1][p2][1]), 1)
        m_tmp = di[p1][p2][0]
        n_nodes = []
        for i,_ in enumerate(m_tmp):
            tmp = f'{m_tmp[i]}'.split(' ')
            print(i, tmp)
            n_nodes.append([int(x.split('=')[1][:-1]) for x in tmp if x[:4]== 'numN'][0])
        df_res_nodes.loc[p1, p2] = np.round(np.mean(n_nodes), 1)

df_res_time.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
df_res_nodes.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')

display(df_res_time)      
display(df_res_nodes)     

0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_a48900d0167a,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_c8e5a8300913,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_6b3707428882,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_e3b304540b8f,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_a6167e8eccb4,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_f8f4edb0d28e,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_46b89e5b7f48,', 'depth=14,', 'numNodes=123,', 'numC

,4,6,8,16,24,32,48
1,9.3,9.2,10.3,15.3,18.8,14.6,16.1
2,6.4,9.5,10.6,13.9,9.4,20.5,12.3
5,10.2,7.4,7.0,7.4,8.2,7.8,11.0
10,11.7,10.7,10.1,8.6,8.1,8.9,9.1
15,15.3,13.5,10.5,8.6,8.2,6.9,9.0
20,15.6,12.9,10.7,8.5,8.1,7.0,9.1
30,22.8,15.9,14.1,10.4,8.6,7.7,10.2
50,32.3,23.4,18.5,12.7,11.7,9.9,12.2
100,55.1,41.0,31.9,23.0,17.2,16.3,17.3


,4,6,8,16,24,32,48
1,123.0,123.0,123.0,123.0,123.0,123.0,123.0
2,101.0,129.0,129.0,149.0,141.0,123.0,127.0
5,87.0,101.0,87.0,77.0,99.0,87.0,97.0
10,69.0,75.0,65.0,61.0,75.0,51.0,67.0
15,69.0,63.0,77.0,51.0,35.0,61.0,53.0
20,35.0,57.0,57.0,39.0,35.0,53.0,73.0
30,49.0,31.0,79.0,61.0,35.0,59.0,63.0
50,73.0,41.0,35.0,47.0,45.0,33.0,41.0
100,57.0,39.0,47.0,41.0,37.0,33.0,39.0


In [7]:
fn = '4cpu_by_16n__4m'

tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
tmp

,Unnamed: 0,4,6,8,16,24,32,48
0,1,9.3,9.2,10.3,15.3,18.8,14.6,16.1
1,2,6.4,9.5,10.6,13.9,9.4,20.5,12.3
2,5,10.2,7.4,7.0,7.4,8.2,7.8,11.0
3,10,11.7,10.7,10.1,8.6,8.1,8.9,9.1
4,15,15.3,13.5,10.5,8.6,8.2,6.9,9.0
5,20,15.6,12.9,10.7,8.5,8.1,7.0,9.1
6,30,22.8,15.9,14.1,10.4,8.6,7.7,10.2
7,50,32.3,23.4,18.5,12.7,11.7,9.9,12.2
8,100,55.1,41.0,31.9,23.0,17.2,16.3,17.3


In [8]:
fn = '4cpu_by_16n__4m'
tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')
tmp

,Unnamed: 0,4,6,8,16,24,32,48
0,1,123.0,123.0,123.0,123.0,123.0,123.0,123.0
1,2,101.0,129.0,129.0,149.0,141.0,123.0,127.0
2,5,87.0,101.0,87.0,77.0,99.0,87.0,97.0
3,10,69.0,75.0,65.0,61.0,75.0,51.0,67.0
4,15,69.0,63.0,77.0,51.0,35.0,61.0,53.0
5,20,35.0,57.0,57.0,39.0,35.0,53.0,73.0
6,30,49.0,31.0,79.0,61.0,35.0,59.0,63.0
7,50,73.0,41.0,35.0,47.0,45.0,33.0,41.0
8,100,57.0,39.0,47.0,41.0,37.0,33.0,39.0
